## **1. flask_login Library Overview**

### **1.1. What is flask_login?**

- **flask_login** is a library that handles user authentication and session management for Flask applications.
- **Key Functionality**:
  - When a user logs in, `flask_login` sends user-related session information to the client via the HTTP response.
  - This session information allows the Flask server to identify and authenticate users during subsequent requests.

### **1.2. Key Operations of flask_login**

### **1.3. Login and Session Creation**

- When a user logs in, their login data is used to create a user object from the `User` class.
- This object is then added to `LoginManager()`, generating a session.
- The Flask server includes this session information in the HTTP response.

### **1.4. `current_user` Object**

- `current_user` is a global proxy provided by `flask_login` that represents the logged-in user.
- **Main Attributes**:
  - `current_user.id`: The user's ID (a Unicode value, equivalent to a Python string).
  - `current_user.is_authenticated`: Boolean indicating whether the user is logged in (`True` or `False`).
  - Additional attributes can be customized by extending the `User` class.

### **1.5. Session Validation**

- For subsequent requests to the server, session information is sent in the HTTP headers.
- If the session's ID matches one managed by `LoginManager()`, APIs protected by `@login_required` are accessible.

### **1.6. Logout Handling**

- Upon user logout, the user's session ID is removed from `LoginManager()`.

### **1.7. Required Support for User Class**

The `User` class should inherit from `UserMixin` and implement the following attributes/methods:
- `is_authenticated`: Returns `True` if the user is logged in.
- `is_active`: Returns `True` if the user's account is active.
- `is_anonymous`: Returns `True` for anonymous (unauthenticated) users.
- `get_id()`: Returns a Unicode value that uniquely identifies the user.

`UserMixin` provides default implementations for these methods, which can be overridden if needed.

## **2. Implementation of flask_login**

### **2.1. Initial Setup**

#### 2.1.1. Define Secret Key

- Flask requires a secret key to securely generate session information:
```python
app.secret_key = os.urandom(24)
```

#### 2.1.2. Initialize LoginManager

- `flask_login` uses `LoginManager` to manage user sessions and enhance security:

In [ ]:
from flask_login import LoginManager

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.session_protection = "strong"  # Optional: Enhances session security

### **2.2. User Session Creation**

- To create a session:
  - Instantiate a user object from the `User` class.
  - Pass the object to `flask_login.login_user()`.
- The session is included in the HTTP response as a `Set-Cookie` header. The browser automatically includes this session in future requests.

#### Example Code:

In [ ]:
from flask_login import login_user

# Create or fetch user object
user = User.create(user_email, blog_id)

# Log in user and create session
login_user(user)

### **2.3. Required Pre-Declared Functions for LoginManager**

#### 2.3.1. User Loader

- Loads the user object from the session information during the first access of `current_user`:

In [ ]:
@login_manager.user_loader
def load_user(user_id):
    return User.get(user_id)

#### 2.3.2. Unauthorized Handler

- Handles unauthorized access to pages protected by `@login_required`:

In [ ]:
from flask import jsonify, make_response

@login_manager.unauthorized_handler
def unauthorized():
    return make_response(jsonify(success=False), 401)

### **2.4. Session Security**

- `flask_login` creates sessions using additional user metadata such as:
  - **IP Address**: Ensures requests originate from the same IP.
  - **User Agent**: Validates the browser or client used to create the session.
- This prevents session hijacking unless both the session and additional metadata are matched.

### **2.5. Protecting Routes**

- Use the `@login_required` decorator to restrict access to logged-in users:

In [ ]:
from flask_login import login_required

@app.route('/protected')
@login_required
def protected():
    return "This is a protected page."

## 3. Implementation of the `User` Class


The `User` class is designed to manage user-related operations. It inherits from the `UserMixin` class to support Flask-Login integration. Below is a detailed explanation of the implementation:

### **3.1. Key Features**

1. **Inheritance from `UserMixin`:**
   - Ensures compatibility with Flask-Login, allowing the class to manage user authentication seamlessly.

2. **Attributes:**
   - `id`: A unique identifier for the user (required).
   - `user_email`: Stores the user's email address.
   - `blog_id`: Associates the user with a specific blog.

3. **Methods:**
   - **`get_id`**: Returns the user ID as a string (required by Flask-Login).
   - **`get` (static method)**: Retrieves a user by their ID from the database.
   - **`find` (static method)**: Retrieves a user by their email address from the database.
   - **`create` (static method)**: Creates a new user if one doesn't already exist for the given email and blog ID.

### **3.2. Code Implementation**

In [ ]:
class User(UserMixin):
    def __init__(self, user_id, user_email, blog_id):
        # Initialize user with ID, email, and blog association
        self.id = user_id
        self.user_email = user_email
        self.blog_id = blog_id

    def get_id(self):
        """
        Returns the user ID as a string, required by Flask-Login.
        """
        return str(self.id)

    @staticmethod
    def get(user_id):
        """
        Retrieve a user by their ID from the database.
        
        Args:
            user_id (int): The unique identifier of the user.
        
        Returns:
            User object if found, otherwise None.
        """
        # Establish database connection
        mysql_db = conn_mysqldb()
        db_cursor = mysql_db.cursor()

        # Query to fetch user by ID
        sql = "SELECT * FROM user_info WHERE USER_ID = '%s'" % str(user_id)
        db_cursor.execute(sql)
        user = db_cursor.fetchone()

        if not user:  # Return None if user is not found
            db_cursor.close()
            return None

        # Create a User object with fetched data
        user = User(user_id=user[0], user_email=user[1], blog_id=user[2])
        db_cursor.close()
        return user

    @staticmethod
    def find(user_email):
        """
        Retrieve a user by their email address from the database.
        
        Args:
            user_email (str): The email of the user.
        
        Returns:
            User object if found, otherwise None.
        """
        # Establish database connection
        mysql_db = conn_mysqldb()
        db_cursor = mysql_db.cursor()

        # Query to fetch user by email
        sql = "SELECT * FROM user_info WHERE USER_EMAIL = '%s'" % str(user_email)
        db_cursor.execute(sql)
        user = db_cursor.fetchone()

        if not user:  # Return None if user is not found
            db_cursor.close()
            return None

        # Create a User object with fetched data
        user = User(user_id=user[0], user_email=user[1], blog_id=user[2])
        db_cursor.close()
        return user

    @staticmethod
    def create(user_email, blog_id):
        """
        Create a new user if one does not already exist for the given email.
        
        Args:
            user_email (str): The email of the user to create.
            blog_id (int): The blog ID to associate with the user.
        
        Returns:
            User object of the newly created or existing user.
        """
        # Check if a user already exists with the given email
        user = User.find(user_email)
        if user is None:  # If user does not exist, create a new one
            mysql_db = conn_mysqldb()
            db_cursor = mysql_db.cursor()

            # Query to insert new user
            sql = "INSERT INTO user_info (USER_EMAIL, BLOG_ID) VALUES ('%s', '%s')" % (
                str(user_email), str(blog_id))
            db_cursor.execute(sql)
            mysql_db.commit()

            # Retrieve and return the newly created user
            return User.find(user_email)
        else:
            # Return the existing user
            return user